In [1]:
from scipy import signal
from scipy.fftpack import fft, ifft
import numpy as np
import math

from IPython.display import Audio 
import soundfile as sf

class DelayLine:
    # create circular buffer
    def __init__(self, max_size):
        self.size = max_size
        self.buffer = np.zeros(max_size, dtype=np.float32)
        self.write_pos = 0

    # write sample to buffer, advance write position
    def write(self, sample):
        self.buffer[self.write_pos] = sample
        self.write_pos = (self.write_pos + 1) % self.size

    # read/tap the delay
    def tap(self, delay_samples):
        read_index = (self.write_pos - delay_samples) % self.size
        return self.buffer[read_index]

class OnePoleLPF:
    def __init__(self, a=0.5):
        self.a = a
        self.y_prev = 0.0

    def process_sample(self, x):
        y = self.a * self.y_prev + (1.0 - self.a) * x
        self.y_prev = y
        return y

class AllPassFilter:
    def __init__(self, delay_samples, g, max_size):
        self.D = delay_samples
        self.g = g
        # use instance of DelayLine circular buffer
        self.delay_line = DelayLine(max_size)

    def process_sample(self, x: float) -> float:
        delayed_value = self.delay_line.tap(self.D)
        y = delayed_value - self.g * x
        new_value = x + self.g * y
        self.delay_line.write(new_value)
        return y

class ModulatedAllPass:
    def __init__(self, max_size: int, g: float, initial_delay: float):
        self.size = max_size
        self.buffer = np.zeros(max_size, dtype=np.float32)
        self.write_pos = 0

        self.g = g
        self.delay = initial_delay 

    def set_delay(self, new_delay: float):
        # update internal delay from modulation
        self.delay = new_delay

    def process_sample(self, x: float) -> float:

        delayed_val = self._fractional_tap(self.delay)
        y = delayed_val - self.g * x
        new_val = x + self.g * y

        # Write new_val into buffer
        self.buffer[self.write_pos] = new_val
        self.write_pos = (self.write_pos + 1) % self.size

        return y

    def tap(self, tap_delay: float) -> float:
        return self._fractional_tap(tap_delay)

    def _fractional_tap(self, delay_samples: float) -> float:
        # Read from buffer, prepare linear interpolation
        d_int = int(np.floor(delay_samples))
        frac = delay_samples - d_int

        base_index = self.write_pos - d_int

        idx_a = base_index % self.size
        idx_b = (base_index - 1) % self.size

        val_a = self.buffer[idx_a]
        val_b = self.buffer[idx_b]

        # Linear interpolation
        val = val_a + frac * (val_b - val_a)
        return val


In [2]:
class EarlyReflectionsChain:
    # Implement early reflection chain
    # create predelay, LPFs, and APFs
    def __init__(self, predelay_samples, lpf_a, apf_params, max_size):
        self.predelay = DelayLine(max_size)
        self.predelay_samples = predelay_samples
        self.lpf1 = OnePoleLPF(a=lpf_a)
        self.allpasses = []
        # Create allpass filters from input params
        for (D, g) in apf_params:
            self.allpasses.append(AllPassFilter(D, g, max_size))

    def process_sample(self, x: float) -> float:
        self.predelay.write(x)
        delayed_sample = self.predelay.tap(self.predelay_samples)

        filtered_sample = self.lpf1.process_sample(delayed_sample)

        # 4 allpasses in series
        ap_out = filtered_sample
        for apf in self.allpasses:
            ap_out = apf.process_sample(ap_out)

        return ap_out
    
class FeedbackDelayChain:
    # Implement feedback delay chain class
    # implement MAPF, delay1, LPF2, AP, delay2 as arguments from relevant classes
    def __init__(self, 
                 mapf, 
                 delay1_samples, 
                 lpf_a,
                 apf_delay, apf_g,
                 delay2_samples,
                 max_size):
        self.mapf = mapf  
        self.delay1 = DelayLine(max_size)
        self.delay1_samples = delay1_samples
        self.lpf2 = OnePoleLPF(lpf_a)
        self.apf5 = AllPassFilter(apf_delay, apf_g, max_size)
        self.delay2 = DelayLine(max_size)
        self.delay2_samples = delay2_samples

    def process_sample(self, x: float) -> float:
        # 1) MAPF
        mapf_out = self.mapf.process_sample(x)

        # 2) Delay1
        self.delay1.write(mapf_out)
        d1_out = self.delay1.tap(self.delay1_samples)

        # 3) LPF2
        lpf2_out = self.lpf2.process_sample(d1_out)

        # 4) APF5
        apf5_out = self.apf5.process_sample(lpf2_out)

        # 5) Delay2
        self.delay2.write(apf5_out)
        d2_out = self.delay2.tap(self.delay2_samples)

        return d2_out
    
def process_full_reverb(x, Fs, N_extra=40000):

    # Create initial reflections
    earlyRef = EarlyReflectionsChain(
        predelay_samples=300,  # your big predelay
        lpf_a=0.9,
        apf_params=[(210, 0.75), (158, 0.75), (561, 0.625), (410, 0.625)],
        max_size=100000  # Must be large enough
    )
    
    # Initialize both feedback/delay chains
    mapf1 = ModulatedAllPass(100000, g=0.7, initial_delay=1343.0)
    chain1 = FeedbackDelayChain(
        mapf=mapf1,
        delay1_samples=6241,
        lpf_a=0.8,
        apf_delay=3931, apf_g=0.5,
        delay2_samples=4681,
        max_size=100000
    )
    
    mapf2 = ModulatedAllPass(100000, g=0.7, initial_delay=995.0)  # or AllPassFilter
    chain2 = FeedbackDelayChain(
        mapf=mapf2,
        delay1_samples=6590,
        lpf_a=0.8,
        apf_delay=2664, apf_g=0.5,
        delay2_samples=5505,
        max_size=100000
    )

    # Parameters
    # Append 0s to add length for reverb tail
    # Individal arrays for each channel
    decay = 0.8  # for example
    length = len(x) + N_extra
    c1_out_arr = np.zeros(length)
    c2_out_arr = np.zeros(length)

    chain2_out_prev = 0.0  # store previous sample from chain2

    # optional LFO for mapf1, mapf2
    lfo_rate = 0.1
    phase1 = 0.0
    phase2 = math.pi  # offset phase
    phase_inc = 2 * math.pi * (lfo_rate / Fs)

    for n in range(length):
        in_sample = x[n] if n < len(x) else 0.0

        # (A) Early reflections
        early_out = earlyRef.process_sample(in_sample)

        # (B) Modulate chain1's MAPF 
        mapf1.set_delay(1343.0 + 12.0 * math.sin(phase1))
        phase1 += phase_inc

        # Summation node for chain1
        c1_in = early_out + decay * chain2_out_prev

        # Process chain1
        c1_out = chain1.process_sample(c1_in)

        # (C) Modulate chain2's MAPF
        mapf2.set_delay(995.0 + 12.0 * math.sin(phase2))
        phase2 += phase_inc

        # Summation node for chain2
        c2_in = early_out + decay * c1_out

        # Process chain2
        c2_out = chain2.process_sample(c2_in)

        # Save
        c1_out_arr[n] = c1_out
        c2_out_arr[n] = c2_out

        # For next iteration, remember c2_out
        chain2_out_prev = c2_out

    return c1_out_arr, c2_out_arr

x, Fs = sf.read("snare.wav")
if x.ndim > 1:
    x = x[:,0]

c1, c2 = process_full_reverb(x, Fs, N_extra=100200)

#   treat chain1 as "Left" and chain2 as "Right" for a stereo output:
stereo_out = np.stack([c1, c2], axis=1)
# Write to WAV - I was having issues with the Audio preview function, my apologies
sf.write("my_reverb_out.wav", stereo_out, Fs)
